In [ ]:
!pip install beautifulsoup4==4.11.1
!pip install cachetools==5.2.0
!pip install certifi==2022.9.24
!pip install charset-normalizer==2.1.1
!pip install google-analytics-data==0.14.2
!pip install google-api-core==2.10.2
!pip install google-auth==2.13.0
!pip install googleapis-common-protos==1.56.4
!pip install grpcio==1.50.0
!pip install grpcio-status==1.50.0
!pip install hvac==1.0.2
!pip install idna==3.4
!pip install proto-plus==1.22.1
!pip install protobuf==4.21.9
!pip install pyasn1==0.4.8
!pip install pyasn1-modules==0.2.8
!pip install pyhcl==0.4.4
!pip install python-dotenv==0.21.0
!pip install requests==2.28.1
!pip install rsa==4.9
!pip install six==1.16.0
!pip install soupsieve==2.3.2.post1
!pip install urllib3==1.26.12

In [ ]:
import requests
import json
import os

In [ ]:
GOOGLE_APPLICATION_CREDENTIALS="[credentials].json"
BASE_URL="[BASE_URL]"
GA_PROPERTY_ID="[GA_PROPERTY_ID]"

In [ ]:
def call_endpoints(url, action, payload):
    endpoint = BASE_URL + url

    if action == 'post':
        response = requests.post(endpoint, data=payload)
    elif action == 'put':
        response = requests.put(endpoint, data=payload)
    else:
        response = requests.get(endpoint, data=payload)

    return response

In [ ]:
def get_data(start_date, end_date=None):
    """Runs a simple report on a Google Analytics 4 property."""

    if not end_date:
        end_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = (end_date + timedelta(days=1))

    end_date = end_date.strftime('%Y-%m-%d')

    client = BetaAnalyticsDataClient()

    request = RunReportRequest(
        property=f"properties/{GA_PROPERTY_ID}",
        dimensions=[Dimension(name="campaignId"),
                    Dimension(name="campaignName"),
                    Dimension(name="dateHour")],
        date_ranges=[DateRange(start_date=start_date, end_date=end_date)],
    )

    response = client.run_report(request)

    results = []

    for row in response.rows:
        r = {
            "campaign_id": row.dimension_values[0].value,
            "campaign_name": row.dimension_values[1].value,
            "date_hour": row.dimension_values[2].value
        }
        results.append(r)

    return results

In [ ]:
end_date = datetime.now()
start_date = (end_date - timedelta(days=1)).strftime('%Y-%m-%d')

sessions = (get_data(start_date, end_date))